Name: suman kumar sahu
Objective:

Data Source:

Status:


spark context is instanciated in sc variable. This is how program communicate with spark. This is older way of entry to spark. For later purpose we will use spark session


In [1]:
rawData=sc.textFile("/home/titan/dataset/ML_JRVI/wine.data")

In [3]:
rawData

/home/titan/dataset/ML_JRVI/wine.data MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [4]:
rawData.take(4)

['1,14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065',
 '1,13.2,1.78,2.14,11.2,100,2.65,2.76,.26,1.28,4.38,1.05,3.4,1050',
 '1,13.16,2.36,2.67,18.6,101,2.8,3.24,.3,2.81,5.68,1.03,3.17,1185',
 '1,14.37,1.95,2.5,16.8,113,3.85,3.49,.24,2.18,7.8,.86,3.45,1480']

In [6]:
from pyspark.mllib.regression import LabeledPoint
def parsepoint(line):
    values=[float(x) for x in line.split(',')]
    return LabeledPoint(values[0],values[1:])


In [7]:
parsedData=rawData.map(parsepoint)
parsedData

PythonRDD[3] at RDD at PythonRDD.scala:53

In [8]:
parsedData.take(5)

[LabeledPoint(1.0, [14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]),
 LabeledPoint(1.0, [13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0]),
 LabeledPoint(1.0, [13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185.0]),
 LabeledPoint(1.0, [14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0]),
 LabeledPoint(1.0, [13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0])]

In [9]:
(trainingData,testData)=parsedData.randomSplit([0.7,0.3])

In [10]:
trainingData

PythonRDD[5] at RDD at PythonRDD.scala:53

In [11]:
from pyspark.mllib.tree import DecisionTree

In [13]:
model= DecisionTree.trainClassifier(trainingData,numClasses=4,
                                   categoricalFeaturesInfo={},
                                   impurity='gini',
                                   maxDepth=3,
                                   maxBins=32)

#numclasses variable must be 1 more than the the value assigned to the training label
#impurity= entropy...similar trees...but...


In [14]:
predictions=model.predict(testData.map(lambda x:x.features))
predictions.take(5)

[1.0, 1.0, 1.0, 1.0, 1.0]

In [25]:
# comparing to the actual label
labelandpredictions=testData.map(lambda lp: lp.label).zip(predictions)
type(labelandpredictions.take(5))
labelandpredictions.take(4)[1]
labelandpredictions.take(5)

[(1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]

In [34]:
# performing Test Accuracy
testAcc=labelandpredictions.filter(lambda lp: lp[0]==lp[1]).count()/float(testData.count())
print('Test Accuracy=',testAcc)
# print('Test Accuracy=', testAcc)

Test Accuracy= 0.9375


In [38]:
from pyspark.mllib.evaluation import MulticlassMetrics
metrics=MulticlassMetrics(labelandpredictions)
metrics.accuracy


0.9375

In [41]:
metrics.fMeasure()

TypeError: fMeasure() missing 1 required positional argument: 'label'

In [43]:
metrics.precision(1.0)

0.8888888888888888

In [44]:
metrics.precision(3)

0.9166666666666666

In [50]:
metrics.confusionMatrix().toArray()
# diagonal element are the correctedly predicted and non- diagonals show the false classification

array([[16.,  0.,  0.],
       [ 2., 18.,  1.],
       [ 0.,  0., 11.]])

In [47]:
metrics.recall(2.0)

0.8571428571428571

In [49]:
help(metrics.confusionMatrix)

Help on method confusionMatrix in module pyspark.mllib.evaluation:

confusionMatrix() method of pyspark.mllib.evaluation.MulticlassMetrics instance
    Returns confusion matrix: predicted classes are in columns,
    they are ordered by class label ascending, as in "labels".
    
    .. versionadded:: 1.4.0



In [51]:
print(model.toDebugString())

DecisionTreeModel classifier of depth 3 with 13 nodes
  If (feature 9 <= 3.9400000000000004)
   If (feature 12 <= 977.5)
    Predict: 2.0
   Else (feature 12 > 977.5)
    If (feature 2 <= 2.5700000000000003)
     Predict: 1.0
    Else (feature 2 > 2.5700000000000003)
     Predict: 2.0
  Else (feature 9 > 3.9400000000000004)
   If (feature 6 <= 1.5950000000000002)
    If (feature 3 <= 16.9)
     Predict: 2.0
    Else (feature 3 > 16.9)
     Predict: 3.0
   Else (feature 6 > 1.5950000000000002)
    If (feature 12 <= 679.0)
     Predict: 2.0
    Else (feature 12 > 679.0)
     Predict: 1.0

